In [1]:
!pip install boto3

                                              0.0/139.9 kB ? eta -:--:--
     ------------------------------         112.6/139.9 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 139.9/139.9 kB 2.1 MB/s eta 0:00:00
                                              0.0/13.6 MB ? eta -:--:--
     -                                        0.4/13.6 MB 12.8 MB/s eta 0:00:02
     ----                                     1.4/13.6 MB 17.2 MB/s eta 0:00:01
     -----                                    1.8/13.6 MB 16.0 MB/s eta 0:00:01
     ------                                   2.4/13.6 MB 13.7 MB/s eta 0:00:01
     ---------                                3.2/13.6 MB 14.5 MB/s eta 0:00:01
     ------------                             4.1/13.6 MB 15.5 MB/s eta 0:00:01
     ---------------                          5.2/13.6 MB 16.7 MB/s eta 0:00:01
     -----------------                        6.0/13.6 MB 16.6 MB/s eta 0:00:01
     --------------------                     6.9/13.6 


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Check if instance is Present....

In [12]:
import boto3
ec2 = boto3.client("ec2", "east-1")
resp = ec2.create_key_pair(KeyName='chriskeyaccess2')
file = open("creds\chriskeyaccess2.pem", "w")
file.write(resp['KeyMaterial'])
file.close()

response = ec2.describe_instances()

instance_name = "cmorrismlops"

instance_name = "sentiment-analysis-mlops"

instance_id = ""

for resp in response["Reservations"]:
    resp = resp["Instances"][0]
    tags = resp.get("Tags",[])

    for tag in tags:
        if tag["Key", ""]== "Name" and tag.get("Value") == instance_name:
            instance_id = resp["InstanceId"]

if instance_id =="":
    print(f"There is no instace found with the name you gave me: {instance_name}")
    ## raise(STOP HERE!!)

instance_id





NoCredentialsError: Unable to locate credentials

## Create an Amazon EC2 instance

In [7]:
import boto3
ec2 = boto3.client("ec2", "east-1")

if instance_id == "":
    response = ec2.run_instances(
        ImageId = 'ami-0a27008020b81eb3a',
        MinCount=1,
        MaxCount = 1,
        InstanceType= "t2.micro",
        KeyName = "chriskeyaccess2",
        BlockDeviceMappings=[
            {
                "DeviceName": "/dev/xvda",
                "Ebs": {
                    "DeleteOnTermination": True,
                    "VolumeSize": 120
                }
                
            }
        ])
    instance_id = response["Instances"][0]["InstanceId"]

    ec2.create_tags(Resources=[instance_id], Tags=[
        {
            'Key': 'Name',
            'Value': instance_name
        }
    ])
    
else:
    print("Instance is already made dumbass.")

    
    

NameError: name 'instance_id' is not defined

## Create Security Group and add rules to it
- Security groups control inbound and outbound traffic of the EC2 instance network interface.
- every EC2 instance must have at least one Security Group associated with it. If no Security Group has been specified during the EC2 instance launch, the default Security Group of the default VPC is associated with the instance.



In [ ]:
group_name = "morristest"

response = ec2.describe_security_groups()

security_group_id = [x['GrouId'] for x in response['SecurityGroups'] if x['GroupName']==group_name]

if security_group_id ==[]: 
    response = ec2.create_security_group(
        GroupName = group_name,
        Description= "Security group for testing"
    )
    security_group_id = response['GroupId']
else:
    security_group_id = security_group_id[0]

security_group_id

In [ ]:
from botocore.exceptions import ClientError

def update_security_group(group_id, protocol, port, cidr):
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId = group_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'TopPort': port,
                    'IpRanges': [{'CidrIp'}]
                }
            ]

        )
    except ClientError as e:
        if e.response['Error']['Code']=="InvalidPermission.Duploicate":
                      print("This rule already exists.")
        else:
            print("an error occurred you idiot!")
            print(e)
            
update_security_group(security_group_id, 'tcp', 22, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 80, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8501, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8502, '0.0.0.0/0')


In [ ]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])



In [ ]:
##Here I am describing what my IAM role will be...

iam = boto3.client("iam")

role_name = "ec2-s3-full-access"

response = iam.get_role(RoleName=role_name)

role_arn = response["Role"]["Arn"]

role_arn

## I will make sure this is an instance profile with the same NAME as the role here....
instance_profile_name = role_name

try:
    iam.get_instance_profile(InstanceProfileName=instance_profile_name)
except iam.exceptions.NoSuchEntityException:
    ## Create an instance profile if one does not exist...
    iam.create_instance_profile(InstanceProfileName=instance_profile_name)
    ## Add role to the instance profile....
    iam.add_role_to_instance_profile(
        InstanceProfileName=instance_profile_name,
        RoleName=role_name
    )

### Now I attach the instance profile to the EC2 instance....
ec2.associate_iam_instance_profile(
    IamInstanceProfile={
        "Name": instance_profile_name
    },
    InstanceId=instance_id
)



### Terminate the Instance here....

In [ ]:
import time


def stop_instances(instance_id):
    print("EC2 Instance Stop")
    ec2.stop_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'stopped')

stop_instances([instance_id])


def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response["Reeservations"][0]["Instances"][0]["State"]["Name"]

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        time.sleep(10)

def terminate_instances(instance_id):
    print("EC2 Institute Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, "terminated")

## terminate_instances([instance_id])







